## **`This is the enhanced version of the code.`**
### **`Includes`**
- *Model Checkpoints && Callbacks*
- *Controlling Overfitting and Underfitting with Dropout, Regularization, BatchNorm etc.*
- *Tensorboard Visuaization*
- *Data Visualization*
- *Model Plotting with keras.utils*
- *HyperParameter Tuning with Keras Tuner*
- *Saving and Loading Model*


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

print(tf.__version__)


In [ ]:
!pip install git+https: // github.com/tensorflow/docs


In [ ]:


import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots


In [ ]:
from IPython import display
from matplotlib import pyplot as plt

import numpy as np

import pathlib
import shutil
import tempfile


In [ ]:
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)


In [ ]:
# Set the paths to the training and testing directories
train_dir = 'path/to/training/directory'
test_dir = 'path/to/testing/directory'

# Set the batch size and image size
batch_size = 32
img_size = (224, 224)


#### **NEED TO SEE IF WE NEED THIS CODE**


In [ ]:

N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(1e4)
BATCH_SIZE = 64
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

# NOTE: Use the Dataset.cache method to ensure that the loader doesn't
#       need to re-read the data from the file on each epoch.

validate_ds = packed_ds.take(N_VALIDATION).cache()
train_ds = packed_ds.skip(N_VALIDATION).take(N_TRAIN).cache()


# Use the Dataset.batch method to create batches of an appropriate size for training.
# Use Dataset.shuffle and Dataset.repeat on the training set.

validate_ds = validate_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)


### **FIXME HERE**

In [ ]:

# Data augmentation to avoid overfitting
# FIXME: Need to replace this `ImageDataGenerator` with `tf.keras.utils.image_dataset_from_directory` as the former is deprocated!
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
# Prepare the data generator for training and testing
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
# Load the pre-trained VGG16 model
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(img_size[0], img_size[1], 3)
)

# Freeze the layers in the pre-trained model
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# Add a few dense layers for classification
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.001,
    decay_steps=STEPS_PER_EPOCH*1000,
    decay_rate=1,
    staircase=False
)


# Compile the model

model.compile(

    # NOTE: Uncomment the optimizer with the schedular.
    # optimizer=tf.keras.optimizers.Adam(lr_schedule),
    optimizer='adam',

    # NOTE: Change the loss function from Categorical Cross Entropy to SCCE.
    loss='categorical_crossentropy',
    # loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),

    metrics=['accuracy']
)

model.summary()


### **Custom Code**


In [ ]:
# NOTE: Add callbacks to facilitate easy model recovery!

callbacks = [
    # The saved model name will include the current epoch.
    keras.callbacks.ModelCheckpoint(
        # Path where to save the model
        filepath="mymodel_{epoch}",
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # ! Only save a model if `val_loss` has improved, this can be problematic!.
        save_best_only=True,

        # the `val_loss` score has improved.
        monitor="val_loss",
        verbose=1,
    )
]
model.fit(
    x_train, y_train, epochs=2, batch_size=64, callbacks=callbacks, validation_split=0.2
)


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator
    # batch_size=64,

)


In [ ]:
print(history.history)


In [ ]:

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

# Save the model
model.save('injury_damage_disaster_classifier.h5')
